[Melanoma Classification : EDA starter](https://www.kaggle.com/parulpandey/melanoma-classification-eda-starter/data)の説明を日本語化しました

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<div align='center'><font size="6" color="#F39C12">SIIM-ISIC Melanoma Classification -EDA </font></div>
<hr>

![](https://impactmelanoma.org/wp-content/uploads/2018/11/Standard-Infographic_0.jpg)
https://impactmelanoma.org/wp-content/uploads/2018/11/Standard-Infographic_0.jpg

メラノーマは、肌の色を決める色素（メラニン）を作るメラノサイトと呼ばれる皮膚細胞から発生する皮膚がんです。メラノーマは、急速に全身に広がるため、皮膚がんの中で最も危険なタイプと考えられていますが、早期に発見されれば、一般的には非常に治療が可能です。
https://www.verywellhealth.com/what-is-melanoma-514215

[Society for Imaging Informatics in Medicine (SIIM)](https://siim.org/page/about_siim)は、医用画像情報学の現在および将来の利用に関心を持つ人々のための主要な医療専門組織です。この学会の使命は、学際的なコミュニティでの教育、研究、イノベーションを通じて、企業全体で医用画像情報学を発展させることにあります。[The International Skin Imaging Collaboration or ISIC](https://siim.org/page/about_siim) Melanoma Projectは、メラノーマの死亡率を減らすために皮膚のデジタル画像処理の応用を促進することを目的とした、学界と産業界のパートナーシップです。

ISICメラノーマプロジェクトの包括的な目標は、メラノーマの早期発見の精度と効率を向上させることで、メラノーマに関連した死亡や不必要な生検を減らす努力を支援することです。

## 目的

このコンテストの目的は、皮膚病変の画像からメラノーマを特定することです。具体的には、同一患者内の画像を用いて、どれがメラノーマを表す可能性が高いかを判断する必要がある。つまり、画像中の病変が悪性か良性かの確率を予測するモデルを作成する必要があります。

## データセット
データセットは、.DIOCOM形式の画像で構成されています。
* DIOCOMフォーマット
* JPEGディレクトリ内のJPEG形式
* tfrecords ディレクトリの TFRecord のフォーマット

また、トレーニング、テスト、提出ファイルからなるメタデータがCSV形式で提供されています。

## 評価指標について

この問題では、我々の提出物は、**area under the ROC curve**を用いて評価されます。ROC曲線（受信機動作特性曲線）とは、すべての分類しきい値における分類モデルの性能を示すグラフです。この曲線は、2つのパラメータをプロットします。

![](https://imgur.com/yNeAG4M.png)

ROC曲線は、異なる分類しきい値でのTPR対FPRをプロットしたものです。分類しきい値を下げると、より多くの項目が陽性として分類され、その結果、偽陽性と真陽性の両方が増加します。次の図は、典型的なROC曲線を示しています。

![](https://imgur.com/N3UOcBF.png)

source: https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc


# 1. 必要なライブラリのインポート

ノートブックをフォークする場合に備えて、インターネットを `ON` モードにしておいてください。

In [ ]:

from os import listdir
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#plotly
!pip install chart_studio
import plotly.express as px
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

import seaborn as sns
sns.set(style="whitegrid")


#pydicom
import pydicom

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')


# Settings for pretty nice plots
plt.style.use('fivethirtyeight')
plt.show()


# 2. 画像データセットの読み込み

In [ ]:
# List files available
print(os.listdir("../input/siim-isic-melanoma-classification"))

In [ ]:
# Defining data path
IMAGE_PATH = "../input/siim-isic-melanoma-classification/"

train_df = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
test_df = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')


#Training data
print('Training data shape: ', train_df.shape)
train_df.head(5)

In [ ]:
#Test data
print('Test data shape: ', test_df.shape)
test_df.head(5)

In [ ]:
train_df.groupby(['benign_malignant']).count()['sex'].to_frame()

# 3. データ探査

## 欠損値

In [ ]:
# Null values and Data types
print('Train Set')
print(train_df.info())
print('-------------')
print('Test Set')
print(test_df.info())

いくつかの列に欠けている値があります。これらは後ほど処理します。

## 画像数

In [ ]:
# Total number of images in the dataset(train+test)
print("Total images in Train set: ",train_df['image_name'].count())
print("Total images in Test set: ",test_df['image_name'].count())

## TrainユニークID

In [ ]:
print(f"The total patient ids are {train_df['patient_id'].count()}, from those the unique ids are {train_df['patient_id'].value_counts().shape[0]} ")

## TestユニークID

In [ ]:
print(f"The total patient ids are {test_df['patient_id'].count()}, from those the unique ids are {test_df['patient_id'].value_counts().shape[0]} ")

固有の患者数は、総患者数よりも少ない。これは、患者が複数のレコードを持っていることを意味します。

In [ ]:
columns = train_df.keys()
columns = list(columns)
print(columns)

## ターゲットカラム探索

In [ ]:
train_df['target'].value_counts()

In [ ]:
train_df['target'].value_counts(normalize=True).iplot(kind='bar',
                                                      yTitle='Percentage', 
                                                      linecolor='black', 
                                                      opacity=0.7,
                                                      color='red',
                                                      theme='pearl',
                                                      bargap=0.8,
                                                      gridcolor='white',
                                                     
                                                      title='Distribution of the Target column in the training set')

## Training男女別分布


In [ ]:
train_df['sex'].value_counts(normalize=True)

In [ ]:
train_df['sex'].value_counts(normalize=True).iplot(kind='bar',
                                                      yTitle='Percentage', 
                                                      linecolor='black', 
                                                      opacity=0.7,
                                                      color='green',
                                                      theme='pearl',
                                                      bargap=0.8,
                                                      gridcolor='white',
                                                     
                                                      title='Distribution of the Sex column in the training set')

## 性別 vs ターゲット

In [ ]:
z=train_df.groupby(['target','sex'])['benign_malignant'].count().to_frame().reset_index()
z.style.background_gradient(cmap='Reds')  

In [ ]:
sns.catplot(x='target',y='benign_malignant', hue='sex',data=z,kind='bar')
plt.ylabel('Count')
plt.xlabel('benign:0 vs malignant:1')

## イメージサイトの場所

In [ ]:
train_df['anatom_site_general_challenge'].value_counts(normalize=True).sort_values()

In [ ]:
train_df['anatom_site_general_challenge'].value_counts(normalize=True).sort_values().iplot(kind='barh',
                                                      xTitle='Percentage', 
                                                      linecolor='black', 
                                                      opacity=0.7,
                                                      color='#FB8072',
                                                      theme='pearl',
                                                      bargap=0.2,
                                                      gridcolor='white',
                                                      title='Distribution of the imaged site in the training set')

## 性別によらないイメージサイトの位置

In [ ]:
z1=train_df.groupby(['sex','anatom_site_general_challenge'])['benign_malignant'].count().to_frame().reset_index()
z1.style.background_gradient(cmap='Reds')

In [ ]:
sns.catplot(x='anatom_site_general_challenge',y='benign_malignant', hue='sex',data=z1,kind='bar')
plt.gcf().set_size_inches(10,8)
plt.xlabel('location of imaged site')
plt.xticks(rotation=45,fontsize='10', horizontalalignment='right')
plt.ylabel('count of melanoma cases')


## 患者さんの年齢分布

In [ ]:
train_df['age_approx'].iplot(kind='hist',bins=30,color='orange',xTitle='Age distribution',yTitle='Count')

## 年齢を視覚化するKDEs
密度プロットでデータを要約し、データの質量がどこにあるかを確認します。[カーネル密度推定プロット](https://chemicalstatistician.wordpress.com/2013/06/09/exploratory-data-analysis-kernel-density-estimation-in-r-on-ozone-pollution-data-in-new-york-and-ozonopolis/)は、単一の変数の分布を示し、平滑化ヒストグラムと考えることができます（これは、各データ点でカーネル（通常はガウス分布）を計算し、すべての個々のカーネルを平均化して、単一の平滑曲線を作成することによって作成されます）。このグラフには、seaborn kdeplotを使用します。

### 年齢の分布（目標値と比較して

In [ ]:
# KDE plot of age that were diagnosed as benign
sns.kdeplot(train_df.loc[train_df['target'] == 0, 'age_approx'], label = 'Benign',shade=True)

# KDE plot of age that were diagnosed as malignant
sns.kdeplot(train_df.loc[train_df['target'] == 1, 'age_approx'], label = 'Malignant',shade=True)

# Labeling of plot
plt.xlabel('Age (years)'); plt.ylabel('Density'); plt.title('Distribution of Ages');

### 男女別の年齢分布

In [ ]:
# KDE plot of age that were diagnosed as benign
sns.kdeplot(train_df.loc[train_df['sex'] == 'male', 'age_approx'], label = 'Male',shade=True)

# KDE plot of age that were diagnosed as malignant
sns.kdeplot(train_df.loc[train_df['sex'] == 'female', 'age_approx'], label = 'Female',shade=True)

# Labeling of plot
plt.xlabel('Age (years)'); plt.ylabel('Density'); plt.title('Distribution of Ages');


## 診断結果の分布

In [ ]:
train_df['diagnosis'].value_counts()

In [ ]:
train_df['diagnosis'].value_counts(normalize=True).sort_values().iplot(kind='barh',
                                                      xTitle='Percentage', 
                                                      linecolor='black', 
                                                      opacity=0.7,
                                                      color='blue',
                                                      theme='pearl',
                                                      bargap=0.2,
                                                      gridcolor='white',
                                                      title='Distribution in the training set')

## Test男女別分布

In [ ]:
test_df['sex'].value_counts(normalize=True)

In [ ]:
test_df['sex'].value_counts(normalize=True).iplot(kind='bar',
                                                      yTitle='Percentage', 
                                                      linecolor='black', 
                                                      opacity=0.7,
                                                      color='green',
                                                      theme='pearl',
                                                      bargap=0.8,
                                                      gridcolor='white',
                                                     
                                                      title='Distribution of the Sex column in the test set')

## Test画像撮影部位

In [ ]:
test_df['anatom_site_general_challenge'].value_counts(normalize=True).sort_values()

In [ ]:
test_df['anatom_site_general_challenge'].value_counts(normalize=True).sort_values().iplot(kind='barh',
                                                      xTitle='Percentage', 
                                                      linecolor='black', 
                                                      opacity=0.7,
                                                      color='#FB8072',
                                                      theme='pearl',
                                                      bargap=0.2,
                                                      gridcolor='white',
                                                      title='Distribution of the imaged site in the test set')

## Test 患者さんの年齢分布

In [ ]:
test_df['age_approx'].iplot(kind='hist',bins=30,color='orange',xTitle='Age distribution',yTitle='Count')

## Test 男女別の年齢分布

In [ ]:
# KDE plot of age that were diagnosed as benign
sns.kdeplot(test_df.loc[test_df['sex'] == 'male', 'age_approx'], label = 'Male',shade=True)

# KDE plot of age that were diagnosed as malignant
sns.kdeplot(test_df.loc[test_df['sex'] == 'female', 'age_approx'], label = 'Female',shade=True)

# Labeling of plot
plt.xlabel('Age (years)'); plt.ylabel('Density'); plt.title('Distribution of Ages');


## 患者のオーバーラップ 
トレーニングセットとテストセットの両方に同じ患者の病変画像が現れないことを確認する必要があります。

In [ ]:
# Extract patient id's for the training set
ids_train = train_df.patient_id.values
# Extract patient id's for the validation set
ids_test = test_df.patient_id.values

# Create a "set" datastructure of the training set id's to identify unique id's
ids_train_set = set(ids_train)
print(f'There are {len(ids_train_set)} unique Patient IDs in the training set')
# Create a "set" datastructure of the validation set id's to identify unique id's
ids_test_set = set(ids_test)
print(f'There are {len(ids_test_set)} unique Patient IDs in the training set')

# Identify patient overlap by looking at the intersection between the sets
patient_overlap = list(ids_train_set.intersection(ids_test_set))
n_overlap = len(patient_overlap)
print(f'There are {n_overlap} Patient IDs in both the training and test sets')
print('')
print(f'These patients are in both the training and test datasets:')
print(f'{patient_overlap}')

# 4. 画像の可視化 . JPEG

## ランダムに選択された画像を可視化する

In [ ]:
images = train_df['image_name'].values

# Extract 9 random images from it
random_images = [np.random.choice(images+'.jpg') for i in range(9)]

# Location of the image dir
img_dir = IMAGE_PATH+'/jpeg/train'

print('Display Random Images')

# Adjust the size of your images
plt.figure(figsize=(10,8))

# Iterate and plot random images
for i in range(9):
    plt.subplot(3, 3, i + 1)
    img = plt.imread(os.path.join(img_dir, random_images[i]))
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    
# Adjust subplot parameters to give specified padding
plt.tight_layout()   

JPEG形式の画像はサイズが異なることがわかります。

## 良性病変の画像の可視化

In [ ]:
benign = train_df[train_df['benign_malignant']=='benign']
malignant = train_df[train_df['benign_malignant']=='malignant']

In [ ]:
images = benign['image_name'].values

# Extract 9 random images from it
random_images = [np.random.choice(images+'.jpg') for i in range(9)]

# Location of the image dir
img_dir = IMAGE_PATH+'/jpeg/train'

print('Display benign Images')

# Adjust the size of your images
plt.figure(figsize=(10,8))

# Iterate and plot random images
for i in range(9):
    plt.subplot(3, 3, i + 1)
    img = plt.imread(os.path.join(img_dir, random_images[i]))
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    
# Adjust subplot parameters to give specified padding
plt.tight_layout()   

## 悪性病変の画像の可視化

In [ ]:
images = malignant['image_name'].values

# Extract 9 random images from it
random_images = [np.random.choice(images+'.jpg') for i in range(9)]

# Location of the image dir
img_dir = IMAGE_PATH+'/jpeg/train'

print('Display malignant Images')

# Adjust the size of your images
plt.figure(figsize=(10,8))

# Iterate and plot random images
for i in range(9):
    plt.subplot(3, 3, i + 1)
    img = plt.imread(os.path.join(img_dir, random_images[i]))
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    
# Adjust subplot parameters to give specified padding
plt.tight_layout()   

## ヒストグラム

ヒストグラムは、画像内で様々な色の値がどのくらいの頻度で発生しているか、つまりピクセルの強度値の頻度をグラフィカルに表したものです。RGB色空間では、ピクセルの値は0から255までの範囲で、0は黒、255は白を表します。ヒストグラムを分析することで、画像の明るさ、コントラスト、強度分布を理解することができます。ここで、各カテゴリからランダムに選択したサンプルのヒストグラムを見てみましょう。

### 良性のカテゴリー

In [ ]:
f = plt.figure(figsize=(16,8))
f.add_subplot(1,2, 1)

sample_img = benign['image_name'][0]+'.jpg'
raw_image = plt.imread(os.path.join(img_dir, sample_img))
plt.imshow(raw_image, cmap='gray')
plt.colorbar()
plt.title('Benign Image')
print(f"Image dimensions:  {raw_image.shape[0],raw_image.shape[1]}")
print(f"Maximum pixel value : {raw_image.max():.1f} ; Minimum pixel value:{raw_image.min():.1f}")
print(f"Mean value of the pixels : {raw_image.mean():.1f} ; Standard deviation : {raw_image.std():.1f}")

f.add_subplot(1,2, 2)

#_ = plt.hist(raw_image.ravel(),bins = 256, color = 'orange',)
_ = plt.hist(raw_image[:, :, 0].ravel(), bins = 256, color = 'red', alpha = 0.5)
_ = plt.hist(raw_image[:, :, 1].ravel(), bins = 256, color = 'Green', alpha = 0.5)
_ = plt.hist(raw_image[:, :, 2].ravel(), bins = 256, color = 'Blue', alpha = 0.5)
_ = plt.xlabel('Intensity Value')
_ = plt.ylabel('Count')
_ = plt.legend(['Red_Channel', 'Green_Channel', 'Blue_Channel'])
plt.show()

### 悪性カテゴリー

In [ ]:
f = plt.figure(figsize=(16,8))
f.add_subplot(1,2, 1)

sample_img = malignant['image_name'][235]+'.jpg'
raw_image = plt.imread(os.path.join(img_dir, sample_img))
plt.imshow(raw_image, cmap='gray')
plt.colorbar()
plt.title('Malignant Image')
print(f"Image dimensions:  {raw_image.shape[0],raw_image.shape[1]}")
print(f"Maximum pixel value : {raw_image.max():.1f} ; Minimum pixel value:{raw_image.min():.1f}")
print(f"Mean value of the pixels : {raw_image.mean():.1f} ; Standard deviation : {raw_image.std():.1f}")

f.add_subplot(1,2, 2)

#_ = plt.hist(raw_image.ravel(),bins = 256, color = 'orange',)
_ = plt.hist(raw_image[:, :, 0].ravel(), bins = 256, color = 'red', alpha = 0.5)
_ = plt.hist(raw_image[:, :, 1].ravel(), bins = 256, color = 'Green', alpha = 0.5)
_ = plt.hist(raw_image[:, :, 2].ravel(), bins = 256, color = 'Blue', alpha = 0.5)
_ = plt.xlabel('Intensity Value')
_ = plt.ylabel('Count')
_ = plt.legend(['Red_Channel', 'Green_Channel', 'Blue_Channel'])
plt.show()

# 5 DIOCOMファイルの前処理 
DICOMは、医用画像を保存・伝送するために最も一般的に使用されており、スキャナー、サーバー、ワークステーション、プリンター、ネットワークハードウェア、複数のメーカーの画像アーカイブ通信システム（PACS）などの医用画像機器を統合することを可能にしています。

DICOM 画像の拡張子は dcm です。DICOMファイルには，ヘッダとデータセットの2つの部分がある。ヘッダは，カプセル化されたデータセットに関する情報を含む。これは、ファイルプリアンブル、DICOMプレフィックス、ファイルメタ要素で構成されています。
幸いなことに、Pydicomと呼ばれるPythonのライブラリがあり、これを使ってDIOCOMファイルを読むことができます。pydicomを使うと、これらの複雑なファイルを簡単に自然なパイソニック構造に読み込んで簡単に操作することができます。pydicomは、これらの複雑なファイルを簡単に自然なピソ構造に読み込んで、簡単に操作できるようにしてくれます。変更したデータセットは、DICOM形式のファイルに再度書き込むことができます。

数年前のコンテストで発表された、DIOCOM画像ファイルへの素晴らしい入門書となる非常に素晴らしい[kernel](https://www.kaggle.com/schlerp/getting-to-know-dicom-and-the-data)があります。
カーネル: https://www.kaggle.com/schlerp/getting-to-know-dicom-and-the-data


In [ ]:
print (pydicom.__version__)

In [ ]:
# https://www.kaggle.com/schlerp/getting-to-know-dicom-and-the-data
def show_dcm_info(dataset):
    print("Filename.........:", file_path)
    print("Storage type.....:", dataset.SOPClassUID)
    print()

    pat_name = dataset.PatientName
    display_name = pat_name.family_name + ", " + pat_name.given_name
    print("Patient's name......:", display_name)
    print("Patient id..........:", dataset.PatientID)
    print("Patient's Age.......:", dataset.PatientAge)
    print("Patient's Sex.......:", dataset.PatientSex)
    print("Modality............:", dataset.Modality)
    print("Body Part Examined..:", dataset.BodyPartExamined)
   
    
    
    if 'PixelData' in dataset:
        rows = int(dataset.Rows)
        cols = int(dataset.Columns)
        print("Image size.......: {rows:d} x {cols:d}, {size:d} bytes".format(
            rows=rows, cols=cols, size=len(dataset.PixelData)))
        if 'PixelSpacing' in dataset:
            print("Pixel spacing....:", dataset.PixelSpacing)

In [ ]:
def plot_pixel_array(dataset, figsize=(5,5)):
    plt.figure(figsize=figsize)
    plt.grid(False)
    plt.imshow(dataset.pixel_array)
    plt.show()
    
i = 1
num_to_plot = 5
for file_name in os.listdir('../input/siim-isic-melanoma-classification/train/'):
        file_path = os.path.join('../input/siim-isic-melanoma-classification/train/',file_name)
        dataset = pydicom.dcmread(file_path)
        show_dcm_info(dataset)
        plot_pixel_array(dataset)
    
        if i >= num_to_plot:
            break
    
        i += 1

## データフレーム内のDIOCOMファイルの情報を抽出する

[Gabriel Preda](https://www.kaggle.com/gpreda) さんが [discussion forum](https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/154658) で以下のコードを共有してくれました。

In [ ]:
# source: https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/154658
folder='train'
PATH='../input/siim-isic-melanoma-classification/'

def extract_DICOM_attributes(folder):
    images = list(os.listdir(os.path.join(PATH, folder)))
    df = pd.DataFrame()
    for image in images:
        image_name = image.split(".")[0]
        dicom_file_path = os.path.join(PATH,folder,image)
        dicom_file_dataset = pydicom.read_file(dicom_file_path)
        study_date = dicom_file_dataset.StudyDate
        modality = dicom_file_dataset.Modality
        age = dicom_file_dataset.PatientAge
        sex = dicom_file_dataset.PatientSex
        body_part_examined = dicom_file_dataset.BodyPartExamined
        patient_orientation = dicom_file_dataset.PatientOrientation
        photometric_interpretation = dicom_file_dataset.PhotometricInterpretation
        rows = dicom_file_dataset.Rows
        columns = dicom_file_dataset.Columns

        df = df.append(pd.DataFrame({'image_name': image_name, 
                        'dcm_modality': modality,'dcm_study_date':study_date, 'dcm_age': age, 'dcm_sex': sex,
                        'dcm_body_part_examined': body_part_examined,'dcm_patient_orientation': patient_orientation,
                        'dcm_photometric_interpretation': photometric_interpretation,
                        'dcm_rows': rows, 'dcm_columns': columns}, index=[0]))
    return df

In [ ]:
extract_DICOM_attributes('train')